In [2]:
import pandas as pd
import numpy as np

In [38]:
def ReadAndPreprocess(path):
    import pandas as pd
    data = pd.read_csv(path)
    data.columns = ['title', 'price', 'sub_title', 'mileage_km', 'year', 'engine_cm3',
                'engine_type', 'city', 'province', 'negotiable']
    data['price'] = data['price'].apply(lambda x: x.replace(" ", "")).astype('int')
    data['mileage_km'] = data['mileage_km'].apply(lambda x: x.replace("km", ""))
    data['mileage_km'] = data['mileage_km'].apply(lambda x: x.replace(" ", "")).astype('int')
    data['engine_cm3'] = data['engine_cm3'].apply(lambda x: x.replace('cm3', ''))
    data['engine_cm3'] = data['engine_cm3'].apply(lambda x: x.replace(' ',''))
    data['province'] = data['province'].apply(lambda x: x.replace('(',''))
    data['province'] = data['province'].apply(lambda x: x.replace(')',''))
    return data

In [15]:
data = pd.read_csv('run_results_21-04-2020.csv')

In [39]:
data = ReadAndPreprocess('run_results_21-04-2020.csv')

In [40]:
data.head()

,title,price,sub_title,mileage_km,year,engine_cm3,engine_type,city,province,negotiable
0,Ford Focus,850,Kombi benzyna klima,210000,2000,1596,Benzyna,Wilkowice,Śląskie,Do negocjacji
1,Audi TT,40500,NaN,40483,2008,Benzyna,NaN,Koszalin,Zachodniopomorskie,Do negocjacji
2,Opel Astra IV,34600,Piękna,67300,2014,1686,Diesel,Wadowice,Małopolskie,NaN
3,Hyundai ix35,42700,"2,0 Benzyna AUTOMAT Nawigacja Klimatronik Pano...",111231,2011,1998,Benzyna,Miechucino,Pomorskie,Do negocjacji
4,Audi A4 2.0,31700,Audi A4 B8 2009r. zarejstrowana,200000,2009,1968,Diesel,Zduńska Wola,Łódzkie,Do negocjacji
